https://www.philipzucker.com/ground_lambda_prolog/

How do we carefully defunctionalize the ground interpreter. The two kinds of cinutations rule/choice point and goal. goal correspond to more familiar continuations

I could make a ground prolog interpreter in prolog. Then that prolog vars stink isn't such a problem.

Prolog continuation passing style
```
add(z,Y,Y).
add(s(X),Y,s(Z)) :- add(X,Y,Z).

add(z,Y,Y,k) :- k.
add(s(X),Y,s(Z),k) :- add(X,Y,Z,k).

bar.
biz.
foo :- bar, biz.

biz(K) :- K. % implciit call(K).
bar(K) :- K.
foo(K) :- bar(biz(K)).


foo :- biz.
foo :- bar.

% refactor
foo :- biz ; bar.

foo(Succ, Fail) :- biz(Succ, bar(Succ, Fail)).


```


```
mi((a,b)) :- mi(a), mi(b).
mi((a;b)) :- mi(a); mi(b).
```


Tabling

Interesting equality. Alpha equiv, beat equiv, simplify, unsat check.

Unification modulo egraphs. Inprocessing in the program database. CHR and Datalog inprocessing. backtrack datalog. KB inprocessing

Ground Lambda Prolog Modulo Egraphs
Proplog modulo Egraphs. We can have higher order equational goals and programs that are dealt with via egraph rewriting.
Sigma, P, E, R |- goal

sig, dbP, kbE, kbR   |- vs, euP, euE, euS

If we combined KB, e-unification, and lambda prolog we'd have these pieces





What makes left and right rules?
(always make progress in sequent. That is part of point. "subformula property" could be relaexed to only contains <= propsitionals according to term ordering
)

What about `=>` as a connective. Rewrite. I mean. `->` as implication is kind of rewrite

Restrict outselves to just flat relations?

What about an extensional type theory that allows => to be put into the type system.


cosignnature. We add to the
coequations
the mu operator is described as time travelling.
pfenning also showed some of them to be write only variables.


Just inlined form of -> and /\
```
a = b  a
-----------
    b

a = b  b
-----------
    a

b |- a   a |- b
----------------
    |- a = b
```


```
a -> b |- b = q
---------------- R=
a -> b |- a = q



a = b |- G
-----------  L->
a -> b |- G


martin lof equality rules?

a = b    p(a)
-------------- E=
    p(b)

p(a) <-> p(b)
-------------- I= ???
    a = b


------------- I=
    a = a

And then we sequentify them to?
What about _just_ a propositional equality connective ~  <->
```


Prolog is more like e-unification where e is the theory of horn clauses or something.

AI magic terms

Constraint outputting form vs constraint threading form.
Proof outputting form

prolog is goal based. So is e-unification. the goal stack could also hold unification info. From the implementation persepctive, finding e-unify narrowing rules and horn rules isn't that different. Syntactic unification and right-sequent-rule goal breakdown are kind of similar. pg 42 of traat
The sharing is kind of a goal trie.



https://citeseerx.ist.psu.edu/document?repid=rep1&type=pdf&doi=9db2053669803a3c14357c47ffdd013f2abd81c4 continuations in lamnbda prolog
https://www.ps.uni-saarland.de/~duchier/python/continuations.html
https://legacy.cs.indiana.edu/ftp/techreports/TR183.pdf haynes logic continuations

yield prolog https://yieldprolog.sourceforge.net/

https://okmij.org/ftp/Computation/monads.html#LogicT


algebraic effects via coroutines

# Equality


Options:
- Focus on rewrite rule in decide?

In [ ]:
def ground_flp(rws, g):
    
    for lhs,rhs in rws:
        if lhs == g:
            return rhs

In [ ]:
from z3 import *

def horn(ps : list[z3.BoolRef], rs : dict[z3.ExprRef, z3.ExprRef], g : z3.BoolRef) -> bool:
    def simp(g):
        while True:
            g1 = substitute(g, *rs.items())
            if g1.eq(g):
                return g
            g = g1

    def right(g):
        if is_true(g):
            return True
        if is_and(g): # R and
            return all(right(c) for c in g.children())
        if is_or(g): # R or
            return any(right(c) for c in g.children())
        elif is_app(g) and not is_true(g) and not is_false(g) and not is_or(g): # decide rule
            return decide(g)

    def decide(g):
        """ Try using one of the axioms"""
        return any(left(p,g) for p in ps)

    def left(p, g):
        if p.eq(g): # initial
            return True
        elif is_and(p):
            return any(left(c, g) for c in p.children())
        elif is_implies(p):
            return left(p.arg(1), g) and right(p.arg(0))
        
    return right(g)

# Defunctionalizing the Ground Interpreter

If we want to do the eager style of unification, we need to expose a stack.


In [ ]:
from z3 import *

def horn(ps : list[z3.BoolRef],g : z3.BoolRef) -> bool:
    def right(g):
        if is_true(g):
            return True
        if is_and(g): # R and
            return all(right(c) for c in g.children())
        if is_or(g): # R or
            return any(right(c) for c in g.children())
        elif is_app(g) and not is_true(g) and not is_false(g) and not is_or(g): # decide rule
            return decide(g)

    def decide(g):
        """ Try using one of the axioms"""
        return any(left(p,g) for p in ps)

    def left(p, g):
        if p.eq(g): # initial
            return True
        elif is_and(p):
            return any(left(c, g) for c in p.children())
        elif is_implies(p):
            return left(p.arg(1), g) and right(p.arg(0))
        
    return right(g)

First convert into conituation passing style

Hmm. Do we return from the continuation or not?
We could return by mutating in the top level continuation.

In [ ]:
from z3 import *

def horn(ps : list[z3.BoolRef],g : z3.BoolRef) -> bool:
    """ program clauses ps and goal g"""
    def right(g, succ, fail):
        #print("right", g)
        if is_true(g):
            return succ(True)
        if is_and(g): # R and
            return all(right(c) for c in g.children())
        if is_or(g): # R or
            return any(right(c) for c in g.children())
        elif is_app(g) and not is_true(g) and not is_false(g) and not is_or(g): # decide rule
            return decide(g)

    def decide(g, succ, fail):
        """ Try using one of the axioms"""
        return any(left(p,g) for p in ps)

    def left(p, g, succ, fail):
        #print("left", p, g)
        if p.eq(g): # initial
            return succ(True)
        elif is_and(p):
            return any(left(c, g) for c in p.children())
        elif is_implies(p):
            return left(p.arg(1), g) and right(p.arg(0))
        
    return right(g)

The only things we add are all and any contexts.
